# Test for CS305

In [1]:
import requests
import base64
import socket
import time
import os

# 所有测试均以client1为例
username = "client1"
password = "123"

AUTHORIZATION = base64.b64encode(bytes(username + ":" + password, "utf-8")).decode(
    "utf-8"
)
print(AUTHORIZATION)

Y2xpZW50MToxMjM=


## Basic Part (80%)

### 1. Basic HTTP Server (20%)

#### 1.1 Persistent Connection (10%)

one socket and many http requests

In [2]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic {AUTHORIZATION}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024)
print(msg.decode())
print("-" * 20)

s.sendall(data.encode())
msg = s.recv(1024)
print(msg.decode())
print("-" * 20)

s.sendall(data.encode())
msg = s.recv(1024)
print(msg.decode())
print("-" * 20)

request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic {AUTHORIZATION}\r\n",
    "Connection: Close\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024)
print(msg.decode())
print("-" * 20)

print('persistent connection is successful')

s.close()

HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:23
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:23
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:23
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:23
Content-Length: 0


--------------------
persistent connection is successful


#### 1.2 Many Clients (5%)

In [3]:
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic {AUTHORIZATION}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s1.connect(("localhost", 8080))

s2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s2.connect(("localhost", 8080))

s3 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s3.connect(("localhost", 8080))

s1.sendall(data.encode())
msg1 = s1.recv(1024)
print(msg1.decode())
print("-" * 20)

s2.sendall(data.encode())
msg2 = s2.recv(1024)
print(msg2.decode())
print("-" * 20)

s3.sendall(data.encode())
msg3 = s3.recv(1024)
print(msg3.decode())
print("-" * 20)

s1.sendall(data.encode())
msg1 = s1.recv(1024)
print(msg1.decode())
print("-" * 20)

s2.sendall(data.encode())
msg2 = s2.recv(1024)
print(msg2.decode())
print("-" * 20)

s3.sendall(data.encode())
msg3 = s3.recv(1024)
print(msg3.decode())
print("-" * 20)

s1.close()
s2.close()
s3.close()

print("many clients is successful")

HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:28
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:28
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:28
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:28
Content-Length: 0


--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-Mi

#### 1.3 Authorization (5%)

In [4]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

# 测试不带认证的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 401 Unauthorized
headers = msg.split("\r\n")
for header in headers:
    if header.lower().startswith("content-length"):
        _1, _2 = header.split(":")

msg = s.recv(int(_2.strip()))
print(msg.decode())
print("-" * 20)

# 测试认证错误的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic aaaaaaaaaa\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 401 Unauthorized
headers = msg.split("\r\n")
for header in headers:
    if header.lower().startswith("content-length"):
        _1, _2 = header.split(":")

msg = s.recv(int(_2.strip()))
print(msg.decode())
print("-" * 20)

# 测试带认证的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic {AUTHORIZATION}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 200 OK
headers = msg.split("\r\n")
for header in headers:
    if header.lower().startswith("set-cookie"):
        _ = header.split(":", 1)[1].strip()
        _ = _.split(";")[0].strip()
        session_id = _.split("=")[1].strip()
        print(f"session_id: {session_id}")

print("-" * 20)
print("AUTHORIZATION is successful")

s.close()

HTTP/1.1 401 Unauthorized
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:34
Content-Type: text/html; charset=utf-8
WWW-Authenticate: Basic realm="Authorization Required"
Content-Length: 224


<html>
    <head><title>Error 401</title></head>
    <body>
        <h1>Error 401</h1>
        <p>Unauthorized No permission -- see authorization schemes</p>
        <p>None</p> 
    </body>
</html>
                        
--------------------
HTTP/1.1 401 Unauthorized
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:34
Content-Type: text/html; charset=utf-8
WWW-Authenticate: Basic realm="Authorization Required"
Content-Length: 248


<html>
    <head><title>Error 401</title></head>
    <body>
        <h1>Error 401</h1>
        <p>Unauthorized No permission -- see authorization schemes</p>
        <p>Invalid authorization format</p> 
    </body>
</html>
                        
--------------------
HTTP/1.1 200 None
Set-Cookie: sess

### 2. View and Download (25%)

#### 2.1 View - No SUSTech-HTTP

In [5]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

endpoints = ["/", "/client1", "/client2"]

for endpoint in endpoints:
    request_line = f"GET {endpoint} HTTP/1.1\r\n"
    headers = [
        "Host: localhost:8080\r\n",
        f"Authorization: Basic {AUTHORIZATION}\r\n",
        "Connection: Keep-Alive\r\n",
    ]
    data = request_line + "".join(headers) + "\r\n"
    s.sendall(data.encode())
    msg = s.recv(2048).decode()
    print(msg, end="")
    headers = msg.split("\r\n")
    for header in headers:
        if header.lower().startswith("content-length"):
            _1, _2 = header.split(":")
            msg = s.recv(int(_2.strip())).decode()
            print(msg)
    print("-" * 20)

s.close()

HTTP/1.1 200 None
Set-Cookie: session-id=951d24c7ae788bb1fb4670200b4efc65; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:10:42
Content-type: text/html
Content-Length: 743


<html>
    <head><title>Index of /</title></head>
    <body>
        <h1>Index of /</h1>
        <ul>
            <li><a href="/?SUSTech-HTTP=0">/</a></li>
            <li><a href="/a.txt?SUSTech-HTTP=1">a.txt</a></li>
            <li><a href="/asserts/?SUSTech-HTTP=0">asserts/</a></li>
            <li><a href="/client1/?SUSTech-HTTP=0">client1/</a></li>
            <li><a href="/html/?SUSTech-HTTP=0">html/</a></li>
            <li><a href="/test/?SUSTech-HTTP=0">test/</a></li>
            <li><a href="/test.txt?SUSTech-HTTP=1">test.txt</a></li>
            <li><a href="/test1.txt?SUSTech-HTTP=1">test1.txt</a></li>
            <li><a href="/user1/?SUSTech-HTTP=0">user1/</a></li>
        </ul>
    </body>
</html>
                    
--------------------
HTTP/1.1 200

#### 2.2 View - SUSTech-HTTP=0

In [6]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

endpoints = ["/", "/client1", "/client2"]

for endpoint in endpoints:
    request_line = f"GET {endpoint}?SUSTech-HTTP=0 HTTP/1.1\r\n"
    headers = [
        "Host: localhost:8080\r\n",
        f"Authorization: Basic {AUTHORIZATION}\r\n",
        "Connection: Keep-Alive\r\n",
    ]
    data = request_line + "".join(headers) + "\r\n"
    s.sendall(data.encode())
    msg = s.recv(2048).decode()
    print(msg, end='')
    headers = msg.split("\r\n")
    for header in headers:
        if header.lower().startswith("content-length"):
            _1, _2 = header.split(":")
            msg = s.recv(int(_2.strip())).decode()
            print(msg)
    print("-" * 20)

s.close()

HTTP/1.1 200 None
Set-Cookie: session-id=32a594ab438c0ad24fa6a7635e8189ce; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:11:29
Content-type: text/html
Content-Length: 743


<html>
    <head><title>Index of /</title></head>
    <body>
        <h1>Index of /</h1>
        <ul>
            <li><a href="/?SUSTech-HTTP=0">/</a></li>
            <li><a href="/a.txt?SUSTech-HTTP=1">a.txt</a></li>
            <li><a href="/asserts/?SUSTech-HTTP=0">asserts/</a></li>
            <li><a href="/client1/?SUSTech-HTTP=0">client1/</a></li>
            <li><a href="/html/?SUSTech-HTTP=0">html/</a></li>
            <li><a href="/test/?SUSTech-HTTP=0">test/</a></li>
            <li><a href="/test.txt?SUSTech-HTTP=1">test.txt</a></li>
            <li><a href="/test1.txt?SUSTech-HTTP=1">test1.txt</a></li>
            <li><a href="/user1/?SUSTech-HTTP=0">user1/</a></li>
        </ul>
    </body>
</html>
                    
--------------------
HTTP/1.1 200

#### 2.3 View - SUSTech-HTTP=1

In [7]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

endpoints = ["/", "/client1", "/client2"]

for endpoint in endpoints:
    request_line = f"GET {endpoint}?SUSTech-HTTP=1 HTTP/1.1\r\n"
    headers = [
        "Host: localhost:8080\r\n",
        f"Authorization: Basic {AUTHORIZATION}\r\n",
        "Connection: Keep-Alive\r\n",
    ]
    data = request_line + "".join(headers) + "\r\n"
    s.sendall(data.encode())
    msg = s.recv(2048).decode()
    print(msg, end='')
    headers = msg.split("\r\n")
    for header in headers:
        if header.lower().startswith("content-length"):
            _1, _2 = header.split(":")
            msg = s.recv(int(_2.strip())).decode()
            print(msg)
    print("-" * 20)

s.close()

HTTP/1.1 200 None
Set-Cookie: session-id=87ea793d948916e79d4e7666c3712351; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:11:41
Content-type: text/html
Content-Length: 86

['a.txt', 'asserts/', 'client1/', 'html/', 'test/', 'test.txt', 'test1.txt', 'user1/']
--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=87ea793d948916e79d4e7666c3712351; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:11:41
Content-type: text/html
Content-Length: 9

['a.txt']
--------------------
HTTP/1.1 404 Not Found
Set-Cookie: session-id=87ea793d948916e79d4e7666c3712351; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:11:41
Content-Type: text/html; charset=utf-8
Content-Length: 231


<html>
    <head><title>Error 404</title></head>
    <body>
        <h1>Error 404</h1>
        <p>Not Found Nothing matches the given URI</p>
        <p>File or directory not

#### 2.4 Download

In [9]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

endpoints = ["/a.txt", "/client1/a.txt", "/client2/a.txt"]

for endpoint in endpoints:
    request_line = f"GET {endpoint} HTTP/1.1\r\n"
    headers = [
        "Host: localhost:8080\r\n",
        f"Authorization: Basic {AUTHORIZATION}\r\n",
        "Connection: Keep-Alive\r\n",
    ]
    data = request_line + "".join(headers) + "\r\n"
    s.sendall(data.encode())
    msg = s.recv(2048).decode()
    print(msg, end='')
    headers = msg.split("\r\n")
    for header in headers:
        if header.lower().startswith("content-length"):
            _1, _2 = header.split(":")
            msg = s.recv(int(_2.strip())).decode()
            print(msg)
    print("-" * 20)

s.close()

HTTP/1.1 200 None
Set-Cookie: session-id=5ed4c21c8e94a54506329ae675a9f0e7; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:21:12
Content-type: text/plain
Content-Length: 19

content for "a.txt"
--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=5ed4c21c8e94a54506329ae675a9f0e7; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:21:12
Content-type: text/plain
Content-Length: 11

sadfsdfasdf
--------------------
HTTP/1.1 404 Not Found
Set-Cookie: session-id=5ed4c21c8e94a54506329ae675a9f0e7; Max-Age=10; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:21:12
Content-Type: text/html; charset=utf-8
Content-Length: 231


<html>
    <head><title>Error 404</title></head>
    <body>
        <h1>Error 404</h1>
        <p>Not Found Nothing matches the given URI</p>
        <p>File or directory not found</p> 
    </body>
</html>
                        
-----

### 3. Upload and Delete (20%)

#### 3.1 Upload (10%)

In [16]:
files = {"firstFile": open('tmp/a.txt', "rb")}


data={}
headers={"Authorization": "Basic Y2xpZW50MToxMjM="}
r=requests.post(url='http://127.0.0.1:8080/upload?path=client1/',data=data,headers=headers, files=files)
print(r) # 200
r=requests.post(url='http://127.0.0.1:8080/upload?path=client2/',data=data,headers=headers, files=files)
print(r) # 403

<Response [200]>
<Response [403]>


#### 3.2 Delete (10%)

In [17]:
url='http://127.0.0.1:8080/delete?path=client1/a.txt'

headers={"Authorization": "Basic Y2xpZW50MToxMjM="}
r=requests.post(url=url, headers=headers)
print(r) # 200

<Response [200]>


### 4. Cookies and Session (10%)

In [34]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

# 测试不带认证的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 401 Unauthorized
headers = msg.split("\r\n")
for header in headers:
    if header.lower().startswith("content-length"):
        _1, _2 = header.split(":")

msg = s.recv(int(_2.strip()))
print(msg.decode())
print("-" * 20)

# 测试认证错误的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic aaaaaaaaaa\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 401 Unauthorized
headers = msg.split("\r\n")
for header in headers:
    if header.lower().startswith("content-length"):
        _1, _2 = header.split(":")

msg = s.recv(int(_2.strip()))
print(msg.decode())
print("-" * 20)

# 测试带认证的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Authorization: Basic {AUTHORIZATION}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 200 OK
headers = msg.split("\r\n")
for header in headers:
    if header.lower().startswith("set-cookie"):
        _ = header.split(":", 1)[1].strip()
        _ = _.split(";")[0].strip()
        session_id = _.split("=")[1].strip()
        print(f"session-id: {session_id}")

print("-" * 20)
print("AUTHORIZATION is successful")
print("-" * 20)

# 测试带cookie的请求
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Cookie: session-id={session_id}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 200 OK
print("-" * 20)

# 我的tocken失效时间为5s
# 等待4s重新请求, 应当成功
# tocken失效时间将重置
time.sleep(4)
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Cookie: session-id={session_id}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 200 OK
print("-" * 20)

# 等待6s重新请求, 失败
time.sleep(6)
request_line = "HEAD / HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    f"Cookie: session-id={session_id}\r\n",
    "Connection: Keep-Alive\r\n",
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg, end="")  # 200 OK
print("-" * 20)

print("cookie is successful")

s.close()

HTTP/1.1 401 Unauthorized
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:51:33
Content-Type: text/html; charset=utf-8
WWW-Authenticate: Basic realm="Authorization Required"
Content-Length: 224


<html>
    <head><title>Error 401</title></head>
    <body>
        <h1>Error 401</h1>
        <p>Unauthorized No permission -- see authorization schemes</p>
        <p>None</p> 
    </body>
</html>
                        
--------------------
HTTP/1.1 401 Unauthorized
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 15:51:33
Content-Type: text/html; charset=utf-8
WWW-Authenticate: Basic realm="Authorization Required"
Content-Length: 248


<html>
    <head><title>Error 401</title></head>
    <body>
        <h1>Error 401</h1>
        <p>Unauthorized No permission -- see authorization schemes</p>
        <p>Invalid authorization format</p> 
    </body>
</html>
                        
--------------------
HTTP/1.1 200 None
Set-Cookie: sess

### 5. Chunked Transfer (5%)

In [3]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 8080))

# 测试不带认证的请求
request_line = "GET /chunked_test.txt?chunked=1 HTTP/1.1\r\n"
headers = [
    "Host: localhost:8080\r\n",
    "Connection: Keep-Alive\r\n",
    f"Authorization: Basic {AUTHORIZATION}\r\n"
]
data = request_line + "".join(headers) + "\r\n"

s.sendall(data.encode())
msg = s.recv(1024).decode()
print(msg)
while msg != '0\r\n\r\n':
    msg = s.recv(1024).decode()
    print(msg, end='')
    
s.close()

HTTP/1.1 200 None
Set-Cookie: session-id=2f94d8404da4c583a4db6313f8daac50; Max-Age=5; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 16:29:50
Content-type: text/plain
Transfer-Encoding: chunked


8
12345678
8
12345678
8
12345678
0



## Bonus (Max 30%)

### 1. Brekpoint Transmission (10%)

In [6]:
#Breakpoint Transmission
url='http://127.0.0.1:8080/client1/a.txt'

data={}
headers={"Authorization": "Basic Y2xpZW50MToxMjM=",
         "Range": "0-1,1-2,2-3"}
r=requests.get(url=url, data=data, headers=headers)
print(r)
print(r.content.decode())

# --THISISMYSELFDIFINEDBOUNDARY
# Content-type= text/plain
# Content-range= 0-1/11

# sa
# --THISISMYSELFDIFINEDBOUNDARY
# Content-type= text/plain
# Content-range= 1-2/11

# ad
# --THISISMYSELFDIFINEDBOUNDARY
# Content-type= text/plain
# Content-range= 2-3/11

# df
# --THISISMYSELFDIFINEDBOUNDARY--

<Response [206]>
--51552928239162499622086d00494d6b
Content-type: text/plain
Content-range: bytes 0-1/28

co
--51552928239162499622086d00494d6b
Content-type: text/plain
Content-range: bytes 1-2/28

on
--51552928239162499622086d00494d6b
Content-type: text/plain
Content-range: bytes 2-3/28

nt
--51552928239162499622086d00494d6b--



### 2. Encryption (10%)

In [140]:
import socket
from cryptography.hazmat.primitives.serialization import load_der_public_key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding as pd_aes
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding as pd_rsa

import requests
import base64

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('localhost', 8080))
print(s)

s.sendall(b'ENCRYPTED')
public_key_bytes = s.recv(1024)
public_key = load_der_public_key(public_key_bytes, backend=default_backend())
print(f'get public_key from server')
print('-'*20)

# AES 密钥和 IV 的设置
key = os.urandom(32)  # AES-256 使用 32 字节密钥
iv = os.urandom(16)   # IV 大小为 16 字节

data = key + b"\r\n" + iv
data = public_key.encrypt(data, pd_rsa.OAEP(mgf=pd_rsa.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None))
s.sendall(data)

msg = s.recv(1024).decode()
print(f"msg from server: {msg}")
print('-'*20)

print("加密测试:")
content = b'abcdefg'
print(f"原文: {content}")

cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()
padder = pd_aes.PKCS7(128).padder()
content = padder.update(content) + padder.finalize()
content = (encryptor.update(content) + encryptor.finalize())
print(f"密文: {content}")
print('-'*20)

s.sendall(b'GET / HTTP/1.1\r\nHost: 127.0.0.1:8080\r\nConnection: keep-alive\r\nAuthorization: Basic Y2xpZW50MToxMjM=\r\n\r\n')

msg = s.recv(2048)
print(msg.decode(), end='')
header = msg.split(b'\r\n')
for header in header:
    header = header.decode()
    if 'Content-Length' in header:
        content_length = int(header.split(':')[1])
        break
content = s.recv(content_length)
content = decryptor.update(content) + decryptor.finalize()
unpadder = pd_aes.PKCS7(128).unpadder()
content = unpadder.update(content) + unpadder.finalize()
print(content.decode(), end='')
print('-'*20)


content = b'--83f4a0c2a7cb010c509229f38bb36ab4\r\nContent-Disposition: form-data; name="firstFile"; filename="upload_ENCRYPTED_test.txt"\r\n\r\ncontent for "upload_ENCRYPTED_test.txt"\r\n--83f4a0c2a7cb010c509229f38bb36ab4--\r\n'
padder = pd_aes.PKCS7(128).padder()
content = padder.update(content) + padder.finalize()
encryptor = cipher.encryptor()
content = (encryptor.update(content) + encryptor.finalize())
s.sendall(b'POST /upload?path=client1/ HTTP/1.1\r\nHost: 127.0.0.1:8080\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nAuthorization: Basic Y2xpZW50MToxMjM=\r\nContent-Length: 160\r\nContent-Type: multipart/form-data; boundary=83f4a0c2a7cb010c509229f38bb36ab4\r\n\r\n'
+content)
msg = s.recv(2048).decode()
print(msg)

s.close()

<socket.socket fd=1640, family=2, type=1, proto=0, laddr=('127.0.0.1', 13177), raddr=('127.0.0.1', 8080)>
get public_key from server
--------------------
msg from server: OK
--------------------
加密测试:
原文: b'abcdefg'
密文: b'\xc5c\x98N3\xdb\x9a\xb6,\xd6\xe7\xf6H2\n9'
--------------------
HTTP/1.1 200 None
Set-Cookie: session-id=1ac96dcbf3e1a053eee968f0ba9e13da; Max-Age=5; Path=/
Server: CS305-2023Fall-PROJ-MiniHttpServer HTTP/1.1
Date: Thu, 28 Dec 2023 17:28:04
Content-type: text/html
Content-Length: 816

<html>
    <head><title>Index of /</title></head>
    <body>
        <h1>Index of /</h1>
        <ul>
            <li><a href="/?SUSTech-HTTP=0">/</a></li>
            <li><a href="/a.txt?SUSTech-HTTP=1">a.txt</a></li>
            <li><a href="/asserts/?SUSTech-HTTP=0">asserts/</a></li>
            <li><a href="/chunked_test.txt?SUSTech-HTTP=1">chunked_test.txt</a></li>
            <li><a href="/client1/?SUSTech-HTTP=0">client1/</a></li>
            <li><a href="/client2/?SUSTech-HTTP=0"